In [1]:
import numpy as np
import pylab as py

%matplotlib inline

In [2]:
# 5. Blind-source separation on sounds

# read in sounds
from sklearn.decomposition import FastICA
import scipy.io.wavfile as wave


#(rate, mario_1_data) = wave.read('back.wav')
#(rate, mario_2_data) = wave.read('speech.wav')
(rate, mario_1_data) = wave.read('test.wav')
(rate, mario_2_data) = wave.read('test_2.wav')
print(rate)

22050


In [3]:
len(mario_2_data)

165172

In [4]:
cut = len(mario_2_data)
cut_mario_1_data = mario_1_data[:cut]

In [5]:
original_X = np.array([cut_mario_1_data, mario_2_data])
original_X=original_X.transpose()
original_X.shape

(165172, 2)

In [6]:
# mix the sounds with the following ratios: 2/1 and 2/3
print('original_X sound dimensions:',original_X.shape)
mixed_X = np.zeros(original_X.shape)

# mixes that shouldn't have to be normalized
mixed_X[:,0] = 2 * cut_mario_1_data + 2 * mario_2_data
mixed_X[:,1] = 1 * cut_mario_1_data + 3 * mario_2_data

# debugging info
#print(bach_data.dtype)   # int16
#print(mixed_X.dtype)     # should be int16

# save the mixes
wave.write('mix1.wav',rate, np.asarray(mixed_X[:,0],dtype=np.int16))
wave.write('mix2.wav',rate, np.asarray(mixed_X[:,1],dtype=np.int16))

original_X sound dimensions: (165172, 2)


In [7]:
# now to unmix! (note, this algorithm knowns NOTHING about the 
# structure of the signal or the amount they were mixed)
ica = FastICA(n_components=2)
unmixed_X = ica.fit_transform(mixed_X)  # Reconstruct signals
calculated_A = ica.mixing_  # Get estimated mixing matrix

In [8]:
# and show and save the results
print('unnormalized mixing matrix:\n',calculated_A)

# renormalize the mixing matrix to be compared with mixing ratios
normed_A = np.zeros((2,2))
normed_A[:,0] = calculated_A[:,0] / calculated_A[0,0] * 2
normed_A[:,1] = calculated_A[:,1] / calculated_A[0,1] * 2
print('normalized mixing matrix\n',normed_A)

wave.write('unmix1.wav',rate, np.asarray(unmixed_X[:,0] / np.max(unmixed_X[:,0]) * 14000,dtype=np.int16) )
wave.write('unmix2.wav',rate, np.asarray(unmixed_X[:,1] / np.max(unmixed_X[:,1]) * 14000,dtype=np.int16))

unnormalized mixing matrix:
 [[-447631.40660658 -889689.08454838]
 [-677961.4938234  -446830.81580844]]
normalized mixing matrix
 [[ 2.          2.        ]
 [ 3.02910602  1.0044651 ]]
